# Limpieza de datos

## Limpieza de movies_dataset

Vamos a iniciar el ETL para los conjuntos de datos que se nos ha proporcionado a la hora de realizar esta actividad. Una vez que finalizamos la limpieza vamos a proceder a exportar estos datos en un formato csv con el propósito de realizar un EDA y finalmente levantarlo en Render o Railways.

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final Individual I\\Datos\\movies_dataset.csv')


C:\Users\User\AppData\Local\Temp\ipykernel_8984\3684129914.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final Individual I\\Datos\\movies_dataset.csv')


Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [3]:
columnas_eliminar = ['adult','homepage','imdb_id','original_title','poster_path','video']

movies = movies.drop(columns=columnas_eliminar)

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [5]:
movies['budget'] = movies['budget'].fillna(value=0)
movies['revenue'] = movies['revenue'].fillna(value=0)

Algo que no me había percatado era que ambas columnas son de tipo object. Ahora las transformo en numéricas con la finalidad de poder seguir trabajando con tranquilidad.

In [6]:
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce').fillna(0)
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce').fillna(0)

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

In [7]:
# Evitamos la división por cero al crear la columna 'return'
movies['return'] = movies.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] != 0 else 0, axis=1)

Los valores nulos del campo release date deben eliminarse.

In [8]:
movies['release_date'] = movies['release_date'].dropna()

Ahora vamos a convertilos en una columna de tipo date

In [9]:
movies['release_date'] = pd.to_datetime(movies['release_date'],errors='coerce',format='%Y-%m-%d')

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [10]:
movies['release_year'] = movies['release_date'].dt.year

Este paso es opcional. Llenamos los nulos con 0

In [11]:
movies['release_year'] = movies['release_year'].fillna(0).astype(int)


In [12]:
movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000,1995
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760,1995
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000,1995


Viendo que se nos pide que creamos una función llamada def cantidad_filmaciones_mes( Mes ) vamos a tener que poner los meses en formato String

In [13]:
#Extraemos el mes y creamos una columna nueva
movies['release_month'] = movies['release_date'].dt.month

#Creamos una columna con el nombre del mes
movies['month_name'] = movies['release_date'].dt.strftime('%B')

In [14]:
movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,release_month,month_name
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,10.0,October
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995,12.0,December
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000,1995,12.0,December
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760,1995,12.0,December
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000,1995,2.0,February


Ahora tenemos que hacer la traducción al español ya que en el ejercicios propuesto se nos pide que este en este idioma.

In [15]:
traslation = {'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'}


movies['month_name_es'] = movies['month_name'].map(traslation)

In [16]:
movies = movies.drop(columns=['month_name'])

In [17]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4494 non-null   object        
 1   budget                 45466 non-null  float64       
 2   genres                 45466 non-null  object        
 3   id                     45466 non-null  object        
 4   original_language      45455 non-null  object        
 5   overview               44512 non-null  object        
 6   popularity             45461 non-null  object        
 7   production_companies   45463 non-null  object        
 8   production_countries   45463 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45466 non-null  float64       
 11  runtime                45203 non-null  float64       
 12  spoken_languages       45460 non-null  object        
 13  s

Ahora me percato que la columna popularity tambien es de tipo object. Voy a tener que pasarla a tipo float para poder trabajar más cómodo

In [18]:
movies['popularity'] =pd.to_numeric(movies['popularity'],errors='coerce')

In [19]:
cant=movies['popularity'].isna().sum()
cant

np.int64(6)

In [20]:
movies = movies.dropna(subset=['popularity'])

Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.


In [21]:
# Mostrar algunos ejemplos de la columna
print(movies['belongs_to_collection'].head(10))


0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
5                                                  NaN
6                                                  NaN
7                                                  NaN
8                                                  NaN
9    {'id': 645, 'name': 'James Bond Collection', '...
Name: belongs_to_collection, dtype: object


Vamos a realizar esta función porque el método explode() no funciona de la manera correcta

In [22]:
def desanidar_belongs(row):
    if pd.isna(row):
        return None,None,None,None
    else:
        try:
            data = eval(row)
            if isinstance(data,dict):
                return data.get('id'),data.get('name'),data.get('poster_path'),data.get('backdrop_path')
            else:
                return None,None,None,None
        except Exception as e:
            print(f'Error en la fila:{row}. Detalle:{e}')
            return None,None,None,None
                
                
                
                
desanidado = movies['belongs_to_collection'].apply(lambda row:pd.Series(desanidar_belongs(row)))

desanidado.columns = ['belongs_to_collection_id', 'belongs_to_collection_name', 'belongs_to_collection_poster_path', 'belongs_to_collection_backdrop_path']

In [23]:
movies = pd.concat([movies,desanidado],axis=1)

movies.drop(columns=['belongs_to_collection'],inplace=True)

Ahora vamos a realizar una función muy parecida pero para la columna genres

In [24]:
def desanidar_genres(row):
    if pd.isna(row):
        return None,None
    else:
        try:
            if pd.isna(row):
                return None,None
            genre_list = eval(row)
            genre_id= [genre['id'] for genre in genre_list]
            genre_names = [genre['name'] for genre in genre_list]
            return genre_id,genre_names
        except Exception as e:
            print(f'Error en la fila_{row}. Detalle:{e}')
            return None,None
        
        
        
desanidado_genre = movies['genres'].apply(lambda row: pd.Series(desanidar_genres(row)))

desanidado_genre.columns = ['genre_id','genre_name']


In [25]:
movies = pd.concat([movies,desanidado_genre],axis=1)

In [26]:
movies.drop(columns=['genres'],inplace=True)

In [27]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45460 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   budget                               45460 non-null  float64       
 1   id                                   45460 non-null  object        
 2   original_language                    45449 non-null  object        
 3   overview                             44506 non-null  object        
 4   popularity                           45460 non-null  float64       
 5   production_companies                 45460 non-null  object        
 6   production_countries                 45460 non-null  object        
 7   release_date                         45376 non-null  datetime64[ns]
 8   revenue                              45460 non-null  float64       
 9   runtime                              45203 non-null  float64       
 10  spoken_language

Vamos a realizar el mismo procedimiento para production_companies

In [28]:
def desanidar_production_companies(row):
    if pd.isna(row):
        return None,None
    else:
        try:
            if pd.isna(row):
                return None,None
            companies_list = eval(row)
            companies_id= [company['id'] for company in companies_list]
            companies_names = [company['name'] for company in companies_list]
            return companies_names,companies_id
        except Exception as e:
            print(f'Error en la fila_{row}. Detalle:{e}')
            return None,None
        
desanidados_companies = movies['production_companies'].apply(lambda row: pd.Series(desanidar_production_companies(row)))

desanidados_companies.columns = ['company_names', 'company_ids']

In [29]:
movies = pd.concat([movies,desanidados_companies],axis=1)

movies.drop(columns=['production_companies'],inplace=True)

In [30]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45460 entries, 0 to 45465
Data columns (total 27 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   budget                               45460 non-null  float64       
 1   id                                   45460 non-null  object        
 2   original_language                    45449 non-null  object        
 3   overview                             44506 non-null  object        
 4   popularity                           45460 non-null  float64       
 5   production_countries                 45460 non-null  object        
 6   release_date                         45376 non-null  datetime64[ns]
 7   revenue                              45460 non-null  float64       
 8   runtime                              45203 non-null  float64       
 9   spoken_languages                     45460 non-null  object        
 10  status         

Ahora con production_countries

In [31]:
def desanidar_production_countries(row):
    if pd.isna(row):
        return None,None
    else:
        try:
            if pd.isna(row):
                return None,None
            country_list = eval(row)
            country_names= [country['name'] for country in country_list]
            country_codes = [country['iso_3166_1'] for country in country_list]
            return country_names,country_codes
        except Exception as e:
            print(f'Error en la fila_{row}. Detalle:{e}')
            return None,None
        
desanidados_countries = movies['production_countries'].apply(lambda row: pd.Series(desanidar_production_countries(row)))

desanidados_countries.columns = ['country_names', 'country_codes']

In [32]:
movies=pd.concat([movies,desanidados_countries],axis=1)

movies.drop(columns=['production_countries'],inplace=True)

In [33]:
movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,status,...,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,genre_id,genre_name,company_names,company_ids,country_names,country_codes
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]",[Pixar Animation Studios],[3],[United States of America],[US]
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,...,NaN,None,None,None,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",[United States of America],[US]
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]","[Warner Bros., Lancaster Gate]","[6194, 19464]",[United States of America],[US]
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,None,None,None,"[35, 18, 10749]","[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[306],[United States of America],[US]
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy],"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",[United States of America],[US]


Ahora lo hacemos con spoken language

In [34]:
def desanidar_spoken_languages(row):
    if pd.isna(row):
        return None,None
    else:
        try:
            if pd.isna(row):
                return None,None
            languages_list = eval(row)
            language_names= [lang['name'] for lang in languages_list]
            language_codes = [lang['iso_639_1'] for lang in languages_list]
            return language_names,language_codes
        except Exception as e:
            print(f'Error en la fila_{row}. Detalle:{e}')
            return None,None
        
desanidados_languages = movies['spoken_languages'].apply(lambda row: pd.Series(desanidar_spoken_languages(row)))

desanidados_languages.columns = ['language_names', 'launguage_codes']

In [35]:
movies = pd.concat([movies,desanidados_languages],axis=1)

movies.drop(columns=['spoken_languages'],inplace=True)

Voy a realizar una ultima eliminación de columnas belongs_to_collection_poster_path y belongs_to_collection_backdrop_path

In [36]:
belong=[ 'belongs_to_collection_poster_path', 'belongs_to_collection_backdrop_path']

movies = movies.drop(columns=belong)

Hemos finalizado momentaneamente la limpieza del dataset Movie. Seguro se va a corregir algunas cosas a medidas que sigamos trabajando.

Vamos a convertir la columna id a una columna de tipo int.

In [37]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce').fillna(0).astype('int64')


## Limpieza de Credits dataset


In [55]:
credits = pd.read_csv('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final Individual I\\Datos\\credits.csv')

In [56]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Podemos observar que tenemos el mismo problemas de anidaciones con este dataset. Vamos a realizar una funcion para desanidar como lo hicimos anteriormente.

In [57]:
def desanidar_cast(row):
    if pd.isna(row):
        return None,None,None,None,None,None,None
    else:
        try:
            if pd.isna(row):
                return None,None,None,None,None,None,None
            cast_list = eval(row)
            cast_names= [cast['name'] for cast in cast_list]
            cast_characters=[cast['character'] for cast in cast_list]
            cast_ids=[cast['id'] for cast in cast_list]
            cast_genders=[cast['gender'] for cast in cast_list]
            cast_orders=[cast['order'] for cast in cast_list]
            cast_profile_path=[cast['profile_path'] for cast in cast_list]
            cast_credit_ids=[cast['credit_id'] for cast in cast_list]
            return cast_names,cast_characters,cast_ids,cast_genders,cast_orders,cast_profile_path,cast_credit_ids
        except Exception as e:
            print(f'Error en la fila_{row}. Detalle:{e}')
            return None,None,None,None,None,None,None
        
desanidados_cast = credits['cast'].apply(lambda row: pd.Series(desanidar_cast(row)))

desanidados_cast.columns = ['cast_names', 'cast_characters', 'cast_ids', 'cast_genders', 'cast_orders', 'cast_profile_paths', 'cast_credit_ids']

In [58]:
credits = pd.concat([credits,desanidados_cast],axis=1)

credits.drop(columns=['cast'],inplace=True)

In [59]:
credits['crew'].value_counts()

crew
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [60]:
def desanidar_crew(row):
    if pd.isna(row):
        return None,None,None,None,None,None,None
    else:
        try:
            if pd.isna(row):
                return None,None,None,None,None,None,None
            crew_list = eval(row)
            crew_names= [crew['name'] for crew in crew_list]
            crew_department=[crew['department'] for crew in crew_list]
            crew_ids=[crew['id'] for crew in crew_list]
            crew_genders=[crew['gender'] for crew in crew_list]
            crew_jobs=[crew['job'] for crew in crew_list]
            crew_profile_path=[crew['profile_path'] for crew in crew_list]
            crew_credit_ids=[crew['credit_id'] for crew in crew_list]
            return crew_names,crew_department,crew_ids,crew_genders,crew_jobs,crew_profile_path,crew_credit_ids
        except Exception as e:
            print(f'Error en la fila_{row}. Detalle:{e}')
            return None,None,None,None,None,None,None
        
desanidados_crew = credits['crew'].apply(lambda row: pd.Series(desanidar_crew(row)))

desanidados_crew.columns = ['crew_names','crew_department','crew_ids','crew_genders','crew_jobs','crew_profile_path','crew_credit_ids']

In [61]:
credits = pd.concat([credits,desanidados_crew],axis=1)

credits.drop(columns=['crew'],inplace=True)

In [62]:
credits.columns

Index(['id', 'cast_names', 'cast_characters', 'cast_ids', 'cast_genders',
       'cast_orders', 'cast_profile_paths', 'cast_credit_ids', 'crew_names',
       'crew_department', 'crew_ids', 'crew_genders', 'crew_jobs',
       'crew_profile_path', 'crew_credit_ids'],
      dtype='object')

Prueba con crew

In [63]:
columns_to_drop = [
    'cast_profile_paths',
    'crew_profile_path',
    'crew_ids',
    'crew_genders',
    'crew_credit_ids'
]

# Eliminar las columnas del DataFrame
credits = credits.drop(columns=columns_to_drop)




In [64]:
credits.columns

Index(['id', 'cast_names', 'cast_characters', 'cast_ids', 'cast_genders',
       'cast_orders', 'cast_credit_ids', 'crew_names', 'crew_department',
       'crew_jobs'],
      dtype='object')

## Union de los datasets

In [65]:
data = pd.merge(movies,credits,left_on='id',right_on='id',how='left')

In [66]:
data.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'return', 'release_year', 'release_month',
       'month_name_es', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'genre_id', 'genre_name', 'company_names',
       'company_ids', 'country_names', 'country_codes', 'language_names',
       'launguage_codes', 'cast_names', 'cast_characters', 'cast_ids',
       'cast_genders', 'cast_orders', 'cast_credit_ids', 'crew_names',
       'crew_department', 'crew_jobs'],
      dtype='object')

Nulos y Nan

In [67]:
# Verificar qué columnas tienen valores nulos
nulos = data.isnull().sum()

# Filtrar las columnas que tienen al menos un valor nulo
columnas_con_nulos = nulos[nulos > 0]

# Mostrar el resumen de columnas con valores nulos
print("Columnas con valores nulos y la cantidad de nulos en cada una:")
print(columnas_con_nulos)

Columnas con valores nulos y la cantidad de nulos en cada una:
original_language                11
overview                        954
release_date                     84
runtime                         257
status                           81
tagline                       25097
release_month                    84
month_name_es                    84
belongs_to_collection_id      41037
belongs_to_collection_name    41037
cast_names                        1
cast_characters                   1
cast_ids                          1
cast_genders                      1
cast_orders                       1
cast_credit_ids                   1
crew_names                        1
crew_department                   1
crew_jobs                         1
dtype: int64


In [68]:
data = data.dropna(subset=['release_date'])

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45452 entries, 0 to 45535
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   budget                      45452 non-null  float64       
 1   id                          45452 non-null  int64         
 2   original_language           45441 non-null  object        
 3   overview                    44511 non-null  object        
 4   popularity                  45452 non-null  float64       
 5   release_date                45452 non-null  datetime64[ns]
 6   revenue                     45452 non-null  float64       
 7   runtime                     45206 non-null  float64       
 8   status                      45372 non-null  object        
 9   tagline                     20425 non-null  object        
 10  title                       45452 non-null  object        
 11  vote_average                45452 non-null  float64       


## Exportacion del dataframe

Lo vamos a exportar en un archivo parquet. La finalidad de este tipo de archivo es para que no haya problemas a la hora de levantar el programa en Render

In [70]:
pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [71]:
import pyarrow as pa
import pyarrow.parquet as pq

In [72]:
tabla = pa.Table.from_pandas(data)
pq.write_table(tabla,'data.parquet')